<a href="https://colab.research.google.com/github/CrisMcode111/DI_Bootcamp/blob/main/w9_d1_Daily_Challenge_MCP_Weather_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Daily Challenge: MCP Weather (Student)
Beginner-friendly MCP server + client (no LLMs). Complete the TODOs, then run the client.

## Setup
Run the install cell. If Colab asks, restart runtime after install.

In [1]:
# Install MCP CLI + SDK
%pip install -qU "mcp[cli]"

In [2]:
# Quick verify
!python --version
!mcp --help | head -n 5

Python 3.12.12
                                                                                
 Usage: mcp [OPTIONS] COMMAND [ARGS]...                                         
                                                                                
 MCP development tools                                                          
                                                                                


## A) Server (server.py)
Implement the WeatherDemo server with one tool and one resource.

In [6]:
%%writefile server.py
import logging
from mcp.server.fastmcp import FastMCP

logging.basicConfig(level=logging.INFO)

mcp = FastMCP("WeatherDemo")

CITY_DATA = {
    "paris": {"temp_c": 21, "condition": "sunny"},
    "london": {"temp_c": 18, "condition": "cloudy"},
    "new york": {"temp_c": 24, "condition": "breezy"},
}

@mcp.tool()
def get_weather(city: str) -> dict:
    """Return simple weather info for a supported city."""
    key = city.strip().lower()
    data = CITY_DATA.get(key)
    if not data:
        return {"error": f"Unsupported city: {city}. Try one of: {', '.join(CITY_DATA)}"}
    logging.info("get_weather called for %s", key)
    return {"city": key, **data}

@mcp.resource("cities://list")
def list_cities() -> str:
    """Return supported cities as newline-separated text."""
    return "\n".join(sorted(CITY_DATA))

if __name__ == "__main__":
    mcp.run()

Overwriting server.py


## B) Client (client.py)
Spawn the server via STDIO, discover capabilities, and call them.

In [4]:

%%writefile client.py
import asyncio
import sys
from pathlib import Path
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Use local server.py with this interpreter to avoid PATH conflicts
SERVER_PATH = Path(__file__).parent / "server.py"
server_params = StdioServerParameters(
    command=sys.executable, args=[str(SERVER_PATH)], env=None
)

def extract_content(payload):
    if hasattr(payload, "contents"):
        contents = payload.contents
        if contents:
            first = contents[0]
            if hasattr(first, "text"):
                return first.text
            if isinstance(first, dict) and "text" in first:
                return first["text"]
            return str(first)
    if hasattr(payload, "content"):
        return payload.content
    return str(payload)


async def run():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            resources = await session.list_resources()
            print("Resources:")
            for res in resources.resources:
                print(f"- {res.uri} ({res.name or ''})")

            tools = await session.list_tools()
            print("Tools:")
            for tool in tools.tools:
                print(f"- {tool.name}")

            cities = await session.read_resource("cities://list")
            print("cities://list ->")
            print(extract_content(cities))

            weather = await session.call_tool("get_weather", {"city": "Paris"})
            print("get_weather(Paris) ->", extract_content(weather))


if __name__ == "__main__":
    asyncio.run(run())


Writing client.py


## C) Run
Single terminal (spawns server):
```
python client.py
```
Or two terminals for debugging:
```
mcp run server.py
python client.py
```
In Colab, just run the next cell.

In [7]:
!python client.py

INFO:mcp.server.lowlevel.server:Processing request of type ListResourcesRequest
Resources:
- cities://list (list_cities)
INFO:mcp.server.lowlevel.server:Processing request of type ListToolsRequest
Tools:
- get_weather
INFO:mcp.server.lowlevel.server:Processing request of type ReadResourceRequest
cities://list ->
london
new york
paris
INFO:mcp.server.lowlevel.server:Processing request of type CallToolRequest
INFO:root:get_weather called for paris
get_weather(Paris) -> [TextContent(type='text', text='{\n  "city": "paris",\n  "temp_c": 21,\n  "condition": "sunny"\n}', annotations=None, meta=None)]


## Troubleshooting
- `mcp: command not found` ? rerun install or restart runtime.
- No tools/resources ? check decorators, restart server.
- City missing ? use one of the supported cities listed by `cities://list`.